In [2]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import cv2, os, math, time, sys
from datetime import timedelta
from sklearn.utils import shuffle
sys.path.append('../../original_images')
from gen_data_batch import generate_batch, generate_batch_1

../../original_images/psvrt.py:3: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/as186233/anaconda2/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/as186233/anaconda2/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/as186233/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/as186233/anaconda2/envs/py36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/as186233/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/kernelapp.py

In [11]:
def see_output_grey(iNp,depth_filter_to_see=0,cmap="gray",figsize=(4,4)):
    img_x = iNp[0,:,:]
    fig = plt.figure(figsize=figsize)
    plt.imshow(img_x, interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
    plt.show()


def see_output(iNp,depth_filter_to_see=0,cmap="gray",figsize=(4,4)):
    img_x = iNp[0,:,:,:]
    fig = plt.figure(figsize=figsize)
    if cmap == "gray":
        plt.imshow(img_x, cmap=plt.get_cmap('gray'))
    else:
        plt.imshow(img_x, interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
    plt.show()
    
def normalise(tensor):
    return tf.div(
   tf.subtract(
      tensor, 
      tf.reduce_min(tensor)
   ), 
   tf.subtract(
      tf.reduce_max(tensor), 
      tf.reduce_min(tensor)
   )
) 

def new_weights(shape, layer_name):
    initializer = tf.contrib.layers.xavier_initializer()
    return tf.Variable(initializer(shape), name=layer_name+'_W')

def new_bias(length, layer_name):
    return tf.Variable(tf.constant(0.05, shape=[length]), name=layer_name+'_b')

def new_conv_layer(input,
                   num_input_channels,
                   filter_size,
                   num_filters,
                   name_scope,
                   layer_name='',
                   use_pooling=True):

    with tf.name_scope(name_scope):
        shape = [filter_size, filter_size, num_input_channels, num_filters]
        weights = new_weights(shape, layer_name)
        biases = new_bias(num_filters, layer_name)

        layer = tf.add(tf.nn.conv2d(input=input, filter=weights, strides=[1,1,1,1], padding='SAME'), biases, name=layer_name)
#         print('layer:', layer)
        if use_pooling:
            layer = tf.nn.max_pool(value=layer,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME', name=layer_name+'_max')
        layer = tf.nn.relu(layer, name=layer_name+'_activation')
        
#         print('maxpooled layer:', layer)
        
    return layer, weights

def flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    
    return layer_flat, num_features

def new_fc_layer(input,
                num_inputs,
                num_outputs,
                name_scope,
                layer_name='',
                use_relu=True):
    
    with tf.name_scope(name_scope):
        weights = new_weights([num_inputs, num_outputs], layer_name)
        biases = new_bias(num_outputs, layer_name)

        layer = tf.add(tf.matmul(input, weights),biases,name=layer_name)
    #     layer = tf.matmul(input, weights) + biases

        if use_relu:
            layer = tf.nn.relu(layer, layer_name+'_activation')
    
    return layer

def argmax_2d(tensor, bat_len):
    
    inp_tensor_shape = tf.shape(tensor)
    nimgs = inp_tensor_shape[0]
    img_dims = inp_tensor_shape[1:]
    img_len = inp_tensor_shape[-1]
    flat_img = tf.reshape(tensor, [-1, tf.reduce_prod(img_dims)])

    # # argmax of the flat tensor
    argmax_x = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) // tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))
    argmax_y = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) % tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))

    res = tf.stack([argmax_x, argmax_y, tf.zeros([bat_len], dtype=tf.float64)], axis = 1)
    
    return res

# def extract_patch(orig_vec, mask_vec):
#     batch_len = tf.shape(mask_vec)[0]
#     input_shape = (img_shape[0], img_shape[1])
#     rows, cols = input_shape[0], input_shape[1]
#     item_shape = tf.Variable([item_size[0], item_size[1]])
#     d_rows, d_cols = item_shape[0], item_shape[1]
#     subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1

#     ii, jj = tf.meshgrid(tf.range(d_rows), tf.range(d_cols), indexing='ij')
#     d_ii, d_jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')

#     subm_ii = ii[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_ii
#     subm_jj = jj[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_jj

#     subm_st = tf.stack([subm_ii, subm_jj], axis=-1)

#     gather_exp = tf.map_fn(lambda mask: tf.gather_nd(mask, subm_st), mask_vec)

#     subm_dims = tf.shape(gather_exp)
#     gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[1] * subm_dims[2], subm_dims[3], subm_dims[4]])
#     reduced_mat = tf.map_fn(lambda mask: tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(mask))[-1], gather_exp)
#     pred_crds = argmax_2d(reduced_mat, batch_len)
    
#     pred_crds = tf.cast(pred_crds, dtype = tf.int64)
    
#     itms = tf.map_fn(lambda idx: tf.cast(tf.slice(orig_vec[tf.cast(idx, tf.int64), :, :, :],pred_crds[tf.cast(idx, tf.int64), :], [item_size[0],item_size[1], 3]), dtype = tf.float64), tf.cast(tf.range(batch_len), dtype = tf.float64))
# #     itms = tf.map_fn(lambda idx: tf.cast(tf.slice(tf.squeeze(vec[idx:idx+1, :, :]), tf.squeeze(pred_crds[idx:idx+1, :]), [item_size[0], item_size[1]]), dtype=tf.int32), tf.range(batch_len))
    
#     return itms


def extract_patch(orig_vec, mask_vec):
    batch_len = tf.shape(mask_vec)[0]
    input_shape = (img_shape[0], img_shape[1])
    rows, cols = input_shape[0], input_shape[1]
    item_shape = tf.Variable([item_size[0], item_size[1]])
    d_rows, d_cols = item_shape[0], item_shape[1]
    subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1

    ii, jj = tf.meshgrid(tf.range(d_rows), tf.range(d_cols), indexing='ij')
    d_ii, d_jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')

    subm_ii = ii[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_ii
    subm_jj = jj[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_jj

    subm_st = tf.stack([subm_ii, subm_jj], axis=-1)

    gather_exp = tf.map_fn(lambda mask: tf.gather_nd(mask, subm_st), mask_vec)

    subm_dims = tf.shape(gather_exp)
    gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[1] * subm_dims[2], subm_dims[3], subm_dims[4]])
    reduced_mat = tf.map_fn(lambda mask: tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(mask))[-1], gather_exp)
    pred_crds = argmax_2d(reduced_mat, batch_len)
    
    pred_crds = tf.cast(pred_crds, dtype = tf.int64, name= "predicted_crds")
    
    itms = tf.map_fn(lambda idx: tf.cast(tf.slice(orig_vec[tf.cast(idx, tf.int64), :, :, :],pred_crds[tf.cast(idx, tf.int64), :], [item_size[0],item_size[1], 3]), dtype = tf.float64), tf.cast(tf.range(batch_len), dtype = tf.float64))
#     itms = tf.map_fn(lambda idx: tf.cast(tf.slice(tf.squeeze(vec[idx:idx+1, :, :]), tf.squeeze(pred_crds[idx:idx+1, :]), [item_size[0], item_size[1]]), dtype=tf.int32), tf.range(batch_len))
    
    return itms

def restore_see_layer(orig, input_name, model_name=None, var_name=None):
    result = []
    with tf.Session('', tf.Graph()) as s:
        with s.graph.as_default():
            if ((model_name != None) and var_name != None):
                saver = tf.train.import_meta_graph(model_name+".meta")
                saver.restore(s, model_name)
                fd = {input_name +':0': orig}
#                 print(fd.shape)
                for var in var_name:
                    var=var+":0"
#                     result = 0

                    result.append(s.run(var, feed_dict=fd))
    return result[0], result[1] 

def test_mask_sd(batch_size, model_mask_sd): 
    np.set_printoptions(suppress=True)
    sd_accuracy = 0.
    mask_accuracy = 0.
    sig_mask_accuracy = 0.
    
    test_set, mask_test_labels, test_labels = generate_batch_1(batch_size, img_shape, item_size, nitems) 
    
#     mask_logits,  sd = restore_see_layer(orig=test_set, input_name = 'x', model_name=model_mask_sd, var_name=['train_mask/mask_6/fc3', 'mask_sd/mask_conv_layer/sd_y_pred'])
    mask_logits, sd = restore_see_layer(orig=test_set, input_name = 'x', model_name=model_mask_sd, var_name=['train_mask/sigmoid_output', 'mask_sd/mask_conv_layer/sd_y_pred'])

#     mask_logits[mask_logits < 0 ] = 0
    
#     print('Predicted Mask Logits: ', mask_logits[0].shape)
#     print('Predicted Mask: ', pred_mask.shape)
#     print('Original Mask: ', mask_test_labels)
#     print('Test SD: ', test_set)

    sd_accuracy = np.sum(np.argmax(sd, axis=1) == np.argmax(test_labels, axis=1))/batch_size

#     cor_prd_imgs = np.sum([True for pm, tm  in zip(pred_mask, mask_test_labels) if np.allclose(pm, tm)])    
#     sig_cor_prd_imgs = np.sum([True for pm, tm  in zip(pred_mask, mask_test_labels) if np.allclose(pm, tm)])    

#     print(cor_prd_imgs)
#     mask_accuracy = cor_prd_imgs/batch_size
#     sig_mask_accuracy = sig_cor_prd_imgs/batch_size
    
#     print('Mask Test Accuracy:{0:>.4f}, Sigmoid Test Accuracy:{1:>.4f}'.format(mask_accuracy, sig_mask_accuracy))

#     print('Mask Test Accuracy:{0:>.4f}, SD Test Accuracy:{1:>.4f}'.format(mask_accuracy, sd_accuracy))
    

    
    return test_set, mask_accuracy, sd_accuracy, mask_logits, mask_test_labels


def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm

def plotNNFilter(units):
    print(units.shape)
    filters = units.shape[3]
    plt.figure(1, figsize=(20,20))
    n_columns = 6
    n_rows = math.ceil(filters / n_columns) + 1
    for i in range(filters):
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(units[0,:,:,i], interpolation="nearest")
        

def getActivations(layer, feed_dict_train):
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        units = session.run(layer, feed_dict=feed_dict_train)
    
        plotNNFilter(units)


In [ ]:
test_bat = 1

# train_data, mask_labels, labels = generate_batch_1(test_bat, img_shape, item_size, nitems) 

feed_dict_train = {x: train_data,
           mask_y_true: mask_labels}

see_output(np.reshape(train_data, [test_bat, img_shape[0], img_shape[1], img_shape[2]]))

# with tf.Session() as session:
#     session.run(tf.global_variables_initializer())
#     units = session.run(num_features, feed_dict=feed_dict_train)

# print(m_num_features)
# getActivations(mask_layer0_conv0, feed_dict_train)
# print("******")
# getActivations(mask_layer1_conv1, feed_dict_train)
# print("******")
# getActivations(mask_layer2_conv2, feed_dict_train)
# print("******")
# getActivations(mask_layer3_conv3, feed_dict_train)
# getActivations(mask_layer4_conv4, feed_dict_train)
getActivations(mask_layer5_conv5, feed_dict_train)
# getActivations(mask_layer_fc3, feed_dict_train)

In [ ]:
test_bat = 1
# test_set, mask_test_labels, test_labels = generate_batch_1(test_bat, img_shape, item_size, nitems) 
# orig_img = np.reshape(test_set, [test_bat, img_shape[0], img_shape[1], img_shape[2]])

# resh_mask_test_lbls = np.reshape(mask_test_labels, [test_bat, img_shape[0], img_shape[1]])

# see_output(orig_img, figsize=(4,4))
# see_output_grey(resh_mask_test_lbls, figsize = (4,4))

orig_img, mask_acc, pred_crds, mask_logits, mask_test_lbls = test_mask_sd(test_bat, single_patch_model)
resh_mask_test_lbls = np.reshape(mask_test_lbls, [test_bat, img_shape[0], img_shape[1]])
resh_orig_img = np.reshape(orig_img, [test_bat, img_shape[0], img_shape[1], img_shape[2]])

resh_logits = np.reshape(mask_logits, [test_bat, img_shape[0], img_shape[1]])
# resh_mask = np.reshape(mask, [test_bat, img_shape[0], img_shape[1]])

print(resh_logits)
# print(list(resh_mask))
# print(pred_crds)
# print(resh_mask[0].shape)
# print(resh_mask_test_lbls)

# print(np.array_equal(np.squeeze(resh_mask[0])), np.squeeze(np.flatten(resh_mask_test_lbls[0]))))

# resh_logits[resh_logits < 0] = 0
see_output(resh_orig_img)
see_output_grey(resh_logits)

# see_output_grey(resh_mask, figsize=(8,8))
see_output_grey(resh_mask_test_lbls)


# resh_mask_test_lbls
# img_x = iNp[0,:,:]
# fig = plt.figure(figsize=(8,8))
# plt.imshow(np.squeeze(resh_mask_test_lbls), interpolation='none', aspect='auto')
#     plt.colorbar(img_x, orientation='horizontal')
# plt.show()

In [12]:
single_patch_model = "SD/single_patch_model.ckpt"
total_imgs = 128
train_batch_size = 64
img_shape = (10,10,3)
item_size = (2,2)
nitems = 2
num_classes = 2

In [13]:
# Placeholders
x = tf.placeholder(tf.float32, shape=[None, img_shape[0]*img_shape[1]*img_shape[2]], name='x')
x_image = tf.reshape(x, [-1, img_shape[0], img_shape[1], img_shape[2]])

mask_y_true = tf.placeholder(tf.float32, shape=[None, img_shape[0] * img_shape[1]], name='mask_y_true')
mask_y_true_cls = tf.argmax(mask_y_true, axis=1) 

sd_y_true = tf.placeholder(tf.float32, shape=[None, 2], name='sd_y_true')
sd_y_true_cls = tf.argmax(sd_y_true, axis=1)        

initializer = tf.contrib.layers.xavier_initializer()

name_scope = 'train_mask'

# mask_graph layer configurations
# mask_graph layer configurations
m_filter_size0 = 3          # Convolution filters(kernel) are 4 x 4 pixels.
m_num_filters0 = 16         # There are 16 of these filters.

m_filter_size1 = 3          # Convolution filters are 4 x 4 pixels.
m_num_filters1 = 32         # There are 16 of these filters.

# Convolutional Layer 2.
m_filter_size2 = 3          # Convolution filters are 2 x 2 pixels.
m_num_filters2 = 64         # There are 16 of these filters.

m_filter_size3 = 3          # Convolution filters are 2 x 2 pixels.
m_num_filters3 = 128         # There are 4 of these filters.

# Convolutional Layer 3.
m_filter_size4 = 3          # Convolution filters are 2 x 2 pixels.
m_num_filters4 = 256         # There are 32 of these filters.

m_filter_size5 = 3          # Convolution filters are 2 x 2 pixels.
m_num_filters5 = 512         # There are 16 of these filters.


# Fully-connected layer.
m_fc_size = 2000             # Number of neurons in fully-connected layer.


"""SD Network Layer Architecture"""
filter_size1 = 4          # Convolution filters are 4 x 4 pixels.
num_filters1 = 16         # There are 16 of these filters.

# Convolutional Layer 2.
filter_size2 = 2          # Convolution filters are 2 x 2 pixels.
num_filters2 = 32         # There are 32 of these filters.

# Convolutional Layer 3.
filter_size3 = 4          # Convolution filters are 2 x 2 pixels.
num_filters3 = 64         # There are 64 of these filters.

# Convolutional Layer 4.
filter_size4 = 2          # Convolution filters are 2 x 2 pixels.
num_filters4 = 128         # There are 128 of these filters.

# Fully-connected layer.
fc_size = 256             # Number of neurons in fully-connected layer.

with tf.name_scope(name_scope):
# First Convolution Layer
    layer_name = 'mask_conv_layer'
    shape = [m_filter_size0, m_filter_size0, img_shape[2], m_num_filters0]
    mask_weights = tf.Variable(initializer(shape), name=layer_name+'_W')  
    mask_biases = tf.Variable(tf.constant(0.05, shape=[m_num_filters0]), name=layer_name+'_b')

    mask_layer0_conv0, weights_conv0 = new_conv_layer(input=x_image,
                                                num_input_channels=img_shape[2],
                                                filter_size=m_filter_size0,
                                                num_filters=m_num_filters0,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv1',
                                                use_pooling=False)
#     layer0_conv0_drpout = tf.nn.dropout(layer0_conv0, 0.3, name="drop_out")

    mask_layer1_conv1, weights_conv1 = new_conv_layer(input=mask_layer0_conv0,
                                                num_input_channels=m_num_filters0,
                                                filter_size=m_filter_size1,
                                                num_filters=m_num_filters1,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv2',
                                                use_pooling=False)

#     layer1_conv1_drpout = tf.nn.dropout(layer1_conv1, 0.3, name="drop_out")

    mask_layer2_conv2, weights_conv2 =  new_conv_layer(input=mask_layer1_conv1,
                                               num_input_channels=m_num_filters1,
                                               filter_size=m_filter_size2,
                                               num_filters=m_num_filters2,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv3',
                                               use_pooling=True)

    mask_layer3_conv3, weights_conv3 =  new_conv_layer(input=mask_layer2_conv2,
                                               num_input_channels=m_num_filters2,
                                               filter_size=m_filter_size3,
                                               num_filters=m_num_filters3,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv4',
                                               use_pooling=False)

    mask_layer4_conv4, weights_conv4 =  new_conv_layer(input=mask_layer3_conv3,
                                               num_input_channels=m_num_filters3,
                                               filter_size=m_filter_size4,
                                               num_filters=m_num_filters4,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv5',
                                               use_pooling=True)


    mask_layer5_conv5, weights_conv5 =  new_conv_layer(input=mask_layer4_conv4,
                                               num_input_channels=m_num_filters4,
                                               filter_size=m_filter_size5,
                                               num_filters=m_num_filters5,
                                                 name_scope = 'mask',
                                                 layer_name = 'conv6',
                                               use_pooling=False)
    
#     layer6_conv6, weights_conv6 =  new_conv_layer(input=layer5_conv5,
#                                            num_input_channels=m_num_filters5,
#                                            filter_size=m_filter_size6,
#                                            num_filters=m_num_filters6,
#                                              name_scope = 'mask',
#                                              layer_name = 'conv7',
#                                            use_pooling=True)

    m_layer_flat, m_num_features = flatten_layer(mask_layer5_conv5)
#     print(layer_flat)
#     layer_fc1 = new_fc_layer(input=layer_flat,
#                              num_inputs=num_features,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc1',
#                              use_relu=False)
#     print('layer_fc1:', layer_fc1)
#     layer_fc2 = new_fc_layer(input=layer_fc1,
#                              num_inputs=m_fc_size,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc2',
#                              use_relu=False)

#     layer_fc3 = new_fc_layer(input=layer_fc2,
#                              num_inputs=m_fc_size,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc3',
#                              use_relu=False)
#     mask_drop_out = tf.nn.dropout(layer_fc3, 0.5, name="drop_out")

    mask_layer_fc3 = new_fc_layer(input=m_layer_flat,
                             num_inputs=m_num_features,
                             num_outputs=img_shape[0] * img_shape[1],
                             name_scope = 'mask',
                             layer_name = 'fc3',
                             use_relu=False)

#     mask_drop_out = tf.nn.dropout(mask_layer_fc4, 0.5, name="drop_out")
#     y_pred = tf.nn.softmax(mask_drop_out, name="softmax_output")
    y_pred = mask_layer_fc3
    
    # round numbers less than 0.5 to zero;
    # by making them negative and taking the maximum with 0
#     differentiable_round = tf.maximum(y_pred-0.499,0)
    # scale the remaining numbers (0 to 0.5) to greater than 1
    # the other half (zeros) is not affected by multiplication
#     differentiable_round = differentiable_round * 10000
    # take the minimum with 1
#     y_pred_round = tf.minimum(differentiable_round, 1)
#     print(differentiable_round)

    y_pred_sigmoid = tf.nn.sigmoid(y_pred, name="sigmoid_output")
    y_pred_sigmoid = tf.reshape(y_pred_sigmoid, [-1, img_shape[0], img_shape[1]])
    
    batch_len = tf.shape(y_pred_sigmoid)[0]
    input_shape = (img_shape[0], img_shape[1])
    rows, cols = input_shape[0], input_shape[1]
    item_shape = tf.Variable([item_size[0], item_size[1]])
    d_rows, d_cols = item_shape[0], item_shape[1]
    subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1

    ii, jj = tf.meshgrid(tf.range(d_rows), tf.range(d_cols), indexing='ij')
    d_ii, d_jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')

    subm_ii = ii[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_ii
    subm_jj = jj[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_jj

    subm_st = tf.stack([subm_ii, subm_jj], axis=-1)
    
    gather_exp = tf.map_fn(lambda mask: tf.gather_nd(mask, subm_st), y_pred_sigmoid)

    subm_dims = tf.shape(gather_exp)
    gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[1] * subm_dims[2], subm_dims[3], subm_dims[4]])
    reduced_mat = tf.map_fn(lambda mask: tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(mask))[-1], gather_exp)
    
    inp_tensor_shape = tf.shape(reduced_mat)
    nimgs = inp_tensor_shape[0]
    img_dims = inp_tensor_shape[1:]
    img_len = inp_tensor_shape[-1]
    flat_img = tf.reshape(reduced_mat, [-1, tf.reduce_prod(img_dims)])

    _, top_idx = tf.nn.top_k(flat_img, nitems)
    topx_idx = tf.squeeze(tf.cast(tf.reshape(top_idx, [-1, 1]), tf.float32) // tf.cast(img_len, tf.float32))
    topy_idx = tf.squeeze(tf.cast(tf.reshape(top_idx, [-1, 1]), tf.float32) % tf.cast(img_len, tf.float32))
    
    res12 = tf.stack([topx_idx, topy_idx, tf.zeros([batch_len * nitems], dtype=tf.float32)], axis = 1, name= "pred_crds12")    
    
    pred_crds12 = tf.cast(res12, dtype = tf.int32, name= "predicted_crds12") 
    pred_crds = tf.map_fn(lambda idx: pred_crds12[idx: idx + 2], tf.range(0, batch_len * nitems, nitems), dtype = tf.int32)
   
  
    items = tf.map_fn(lambda img_idx: tf.reshape(tf.map_fn(lambda itm_idx: tf.slice(x_image[img_idx, :, :, :],
                                                      pred_crds[img_idx, itm_idx, :], 
                                                      [item_size[0],item_size[1], 3])
                                               , tf.range(nitems), dtype = tf.float32), 
                                                [item_size[0] * 2, item_size[0], 3])
                         , tf.range(batch_len), dtype = tf.float32)

#     mask_loss = tf.square(mask_y_true - y_pred)
#     mask_cost = tf.reduce_mean(mask_loss)
    
#     train_op1 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(mask_loss)
    
    ### some more performance measures
#     mask_correct_prediction = tf.equal(y_pred, mask_y_true)
#     mask_accuracy = tf.reduce_mean(tf.cast(mask_correct_prediction, tf.float32))        

name_scope = 'mask_sd'
with tf.name_scope(name_scope):

    input_sd = tf.cast(items, dtype = tf.float32, name=layer_name + "/input_sd")
    
    layer1_conv1, weights_conv1 = new_conv_layer(input=input_sd,
                                                num_input_channels=img_shape[2],
                                                filter_size=filter_size1,
                                                num_filters=num_filters1,
                                                 name_scope = 'mask_sd_graph',
                                                 layer_name = 'conv1',
                                                use_pooling=True)

    layer2_conv2, weights_conv2 = new_conv_layer(input=layer1_conv1,
                                                num_input_channels=num_filters1,
                                                filter_size=filter_size2,
                                                num_filters=num_filters2,
                                                 name_scope = 'mask_sd_graph',
                                                 layer_name = 'conv2',
                                                use_pooling=True)

    layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
                                               num_input_channels=num_filters2,
                                               filter_size=filter_size3,
                                               num_filters=num_filters3,
                                                 name_scope = 'mask_sd_graph',
                                                 layer_name = 'conv3',
                                               use_pooling=True)

    layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
                                               num_input_channels=num_filters3,
                                               filter_size=filter_size4,
                                               num_filters=num_filters4,
                                                 name_scope = 'mask_sd_graph',
                                                 layer_name = 'conv4',
                                               use_pooling=True)

    layer_flat, num_features = flatten_layer(layer4_conv4)       

    layer_fc1 = new_fc_layer(input=layer_flat,
                     num_inputs=num_features,
                     num_outputs=fc_size,
                     name_scope = 'mask_sd_graph',
                     layer_name = 'fc1',
                     use_relu=True)

    layer_fc2 = new_fc_layer(input=layer_fc1,
                             num_inputs=fc_size,
                             num_outputs=fc_size,
                             name_scope = 'mask_sd_graph',
                             layer_name = 'fc2',
                             use_relu=False)

    layer_fc3 = new_fc_layer(input=layer_fc2,
                             num_inputs=fc_size,
                             num_outputs=fc_size,
                             name_scope = 'mask_sd_graph',
                             layer_name = 'fc3',
                             use_relu=False)

    layer_fc4 = new_fc_layer(input=layer_fc3,
                             num_inputs=fc_size,
                             num_outputs=num_classes,
                             name_scope = 'mask_sd_graph',
                             layer_name = 'fc4',
                             use_relu=False)


    
    
    sd_y_pred = tf.nn.softmax(layer_fc4, name=layer_name + "/sd_y_pred")
    sd_y_pred_cls = tf.argmax(sd_y_pred, axis=1)
    
    sd_loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=sd_y_pred, labels=sd_y_true)
    
#     train_op1 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(mask_loss)
    train_op2 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(sd_loss)
#     final_train_op = tf.group(train_op1, train_op2)    

    sd_cost = tf.reduce_mean(sd_loss)
    
    sd_correct_prediction = tf.equal(sd_y_pred_cls, sd_y_true_cls)
    sd_accuracy = tf.reduce_mean(tf.cast(sd_correct_prediction, tf.float32))        
    

In [16]:
def optimize(num_epochs, save_model=True,save_name= "base_model",restore_model=False,restore_name=None):
    total_iterations = 0
    done_train_imgs = 0
    start_time = time.time()
    start_batch=0
    end_batch = train_batch_size
    test_batch_size = 256
    plot_accuracy=[]
    plot_accuracy_epoch=[]
    plot_training_size=[]
    plot_training_size_epoch=[]
    plot_mask = []
    plot_mask_sd = []
    plot_sd = []
    plot_sr = []
    saver = tf.train.Saver()
    
    sum_accuracy = 0.0
    n = 1

    session = tf.Session()         
    session.run(tf.global_variables_initializer())
    
            #to save the model
    for i in range(0, num_epochs):   
        start_batch=0
        end_batch = train_batch_size

        print("Epoch:", i + 1)

        if restore_model==True:
            if restore_name==None:
                print("No model file specified")
                return
            else:
                saver.restore(session,restore_name)

        sum_accuracy = 0.0
        n = 1
        while end_batch < total_imgs:

            train_data, mask_labels, labels = generate_batch_1(train_batch_size, img_shape, item_size, nitems) 
#             print(train_data.shape)
#             print(train_data[0])
            if not len(train_data) and not len(labels) and not len(mask_labels):
                print("All images have been processed.")
                break;

            feed_dict_train = {x: train_data,
                              sd_y_true: labels}

#             feed_dict_train = {x: train_data,
#                        mask_y_true: mask_labels}

            session.run(train_op2, feed_dict=feed_dict_train)

            sd_acc, sd_co = session.run([sd_correct_prediction, sd_cost], feed_dict=feed_dict_train)            
            msg = "Optimization Iteration: {0:>6},  SD Loss: {1:>.4f}"

#             msg = "Optimization Iteration: {0:>6}, SD Training Accuracy: {1:>6.1%}, SD Loss: {2:>.4f}"
            print(msg.format(end_batch + 1,  sd_co))

#             sd_acc, sd_co, itms = session.run([sd_correct_prediction, sd_cost, items], feed_dict=feed_dict_train)            
#             msg = "Optimization Iteration: {0:>6}, SD Training Accuracy: {1:>6.1%}, Mask Loss: {2:>.4f}"
#             print(msg.format(end_batch + 1, sd_acc, sd_co))

#             print(list(train_data[0]))
#             print('Predicted Cords: ', list(pred_crd))

#             fig=plt.figure(figsize=(8, 8))
#             columns = 8
#             rows = 8
#             for i in range(1, columns*rows):
#                 img = itms[i]
#                 fig.add_subplot(rows, columns, i)
#                 plt.imshow(img)
#             plt.show()            
            
            start_batch += train_batch_size
            end_batch += train_batch_size
        if save_model==True:
            if save_name==None:
                print("No model specified, model not being saved")
                return
            else:
                save_path = saver.save(session, save_name)
                restore_model = True
                print("Model saved in file: %s" % save_name)
    
        s_time = time.time()                    
        _, mask_acc, sd_acc, _, _ = test_mask_sd(test_batch_size, save_name)
        print('Test Accuracy:', sd_acc)
#         plot_mask.append(mask_acc)
        plot_sd.append(sd_acc)
        e_time = time.time()
        time_dif = e_time - s_time
        print("Test Mask SD Time usage: " + str(timedelta(seconds=int(round(time_dif)))))  

    end_time = time.time()
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif))))) 
    print('plot mask accuracy:')
#     print(plot_mask)
    print(plot_sd)


In [15]:
save_model = True
save_name = single_patch_model
restore_model=False
restore_name=single_patch_model
optimize(num_epochs=10, save_model=True,save_name=single_patch_model,restore_model=restore_model,restore_name=single_patch_model)




Epoch: 1


../../original_images/psvrt.py:253: RuntimeWarning: divide by zero encountered in long_scalars
  running_orientation += np.arctan(y / x)


Optimization Iteration:     65,  SD Loss: 0.6892
Model saved in file: SD/single_patch_model.ckpt
INFO:tensorflow:Restoring parameters from SD/single_patch_model.ckpt
Test Mask SD Time usage: 0:00:01
Epoch: 2
INFO:tensorflow:Restoring parameters from SD/single_patch_model.ckpt
Optimization Iteration:     65,  SD Loss: 0.6973
Model saved in file: SD/single_patch_model.ckpt
INFO:tensorflow:Restoring parameters from SD/single_patch_model.ckpt
Test Mask SD Time usage: 0:00:01
Epoch: 3
INFO:tensorflow:Restoring parameters from SD/single_patch_model.ckpt
Optimization Iteration:     65,  SD Loss: 0.6939
Model saved in file: SD/single_patch_model.ckpt
INFO:tensorflow:Restoring parameters from SD/single_patch_model.ckpt
Test Mask SD Time usage: 0:00:01
Epoch: 4
INFO:tensorflow:Restoring parameters from SD/single_patch_model.ckpt
Optimization Iteration:     65,  SD Loss: 0.6923
Model saved in file: SD/single_patch_model.ckpt
INFO:tensorflow:Restoring parameters from SD/single_patch_model.ckpt
Tes

In [ ]:
# print(train_data.shape)
session = tf.Session()         
session.run(tf.global_variables_initializer())

feed_dict_train = {x: train_data,
           mask_y_true: mask_labels}

session.run(final_train_op, feed_dict=feed_dict_train)
mask_acc, mask_co, sd_acc, sd_co = session.run(, )            
msg = "Optimization Iteration: {0:>6}, Mask Training Accuracy: {1:>6.1%}, Mask Loss: {2:>.4f}, SD Training Accuracy: {1:>6.1%}, SD Loss: {2:>.4f}"
print(msg.format(1, mask_acc, mask_co, sd_acc, sd_co))

# print(session.run(tf.squeeze([y_pred_norm]), feed_dict=feed_dict_train))
# print(session.run(tf.squeeze([items]), feed_dict=feed_dict_train))

# print(np.array(s_ii))
# print(np.array(s_jj))

In [ ]:
def get_tensors(graph=tf.get_default_graph()):
    return [t for op in graph.get_operations() for t in op.values()]
get_tensors()

### Gand Balaaa!

In [ ]:
item_size = (3,3)
"""
This section of code extracts a Single patch from an input image using convolution logic. Additionally, it should be dynamic!

"""

# z_vec = tf.zeros([5,5])
# z_vec[:, ]

vec1 = tf.constant(np.array([[0,0,0,0,0],
                             [0,0,0,0,0],
                             [0,1,1,1,0],
                             [0,1,1,1,0],
                             [0,1,1,1,0]]), dtype=tf.float32)
# vec1 = tf.constant(np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]), dtype=tf.float32)

# vec1 = tf.constant(np.array([[0,0,0,0],[1,1,0,0],[1,1,0,0],[0,0,0,0]]))
# vec = tf.stack([vec1, vec2])
input_shape = tf.shape(vec1)
rows, cols = input_shape[0], input_shape[1]
item_shape = tf.Variable([item_size[0], item_size[1]])
d_rows, d_cols = item_shape[0], item_shape[1]
subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1

ii, jj = tf.meshgrid(tf.range(d_rows), tf.range(d_cols), indexing='ij')
d_ii, d_jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')

subm_ii = ii[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_ii
subm_jj = jj[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_jj

subm_st = tf.stack([subm_ii, subm_jj], axis=-1)

gather_exp = tf.gather_nd(vec1, subm_st)
subm_dims = tf.shape(gather_exp)
gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[0] * subm_dims[1], subm_dims[2], subm_dims[3]])
reduced_mat = tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(gather_exp))[-1]




# res = argmax_2d(reduced_mat)
# print(reduced_mat)
# pred_crds = tf.argmax(reduced_mat, axis = -1)
# slice_exp = tf.slice(vec1, pred_crds[:2], [item_size[0],item_size[1]])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(np.array(sess.run([vec1])))
#     print(sess.run([ii, jj]))
#     print(sess.run([d_ii, d_jj]))    
#     print(sess.run([subm_ii, subm_jj]))
#     print('subm_st: ', sess.run(subm_st))
    print(sess.run([reduced_mat]))
#     print(sess.run([res]))
    

In [ ]:
"""
This section of code extracts a Single patch from an input image using convolution logic.

"""

# z_vec = tf.zeros([5,5])
# z_vec[:, ]

# vec1 = tf.constant(np.array([[1,1,0,0,0],[1,1,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]), dtype=tf.float32)
# vec1 = tf.constant(np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]), dtype=tf.float32)

# vec1 = tf.constant(np.array([[0,0,0,0],[1,1,0,0],[1,1,0,0],[0,0,0,0]]))
# vec = tf.stack([vec1, vec2])
# input_shape = tf.shape(vec1)
# rows, cols = input_shape[0], input_shape[1]
# item_shape = tf.Variable([item_size[0], item_size[1]])
# d_rows, d_cols = item_shape[0], item_shape[1]
# subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1

# ii, jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')
# d_ii, d_jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')

# subm_ii = ii[:subm_rows - 1, :subm_cols - 1, tf.newaxis, tf.newaxis] + d_ii
# subm_jj = jj[:subm_rows - 1, :subm_cols - 1, tf.newaxis, tf.newaxis] + d_jj

# subm_st = tf.stack([subm_ii, subm_jj], axis=-1)

# gather_exp = tf.gather_nd(vec1, subm_st)
# subm_dims = tf.shape(gather_exp)
# gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[0] * subm_dims[1], subm_dims[2], subm_dims[3]])
# reduced_mat = tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(gather_exp))[-1]
# res = argmax_2d(reduced_mat)

vec = tf.constant(np.array([[0.06943432, 0.07020187, 0.06474971, 0.06497546, 0.07147662, 0.07259881,
  0.05938388, 0.05955945, 0.06461262],
                            [0.0666275,  0.07140117, 0.06605807, 0.0650783,  0.07378063, 0.07307022,
  0.05833262, 0.059543,   0.06464229],
 [0.06950475, 0.07092812, 0.06614418, 0.06343528, 0.07138074, 0.07305786,
  0.05641314, 0.05814052, 0.06446858],
 [0.06943432, 0.07020187, 0.06474971, 0.06497546, 0.07147662, 0.07259881,
  0.05938388, 0.05955945, 0.06461262]]))

_, top_idx = tf.nn.top_k(vec, 1)
tf.argmax(vec)

# slice_exp = tf.slice(vec1, pred_crds[:2], [item_size[0],item_size[1]])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(np.array(sess.run([top_idx])))
    print(sess.run(tf.argmax(vec)))


In [ ]:
train_batch_size = 2
img_shape = (4,4,3)
item_size = (2,2)
nitems = 1
num_classes = 2
train_data, mask_labels, labels = generate_batch_1(train_batch_size, img_shape, item_size, nitems)



print(train_data.shape)
print(mask_labels)

In [ ]:
# item_size = (2,2)

"""
This section of code extracts a Single patch from a batch of input images using convolution logic.

"""

# vec1 = tf.constant([[0.5000534,  0.50027275, 0.5007929,  0.5007636 ],
#   [0.49887955, 0.49952933, 0.7321372,  0.73167676],
#   [0.49823606, 0.49942428, 0.73225975, 0.7310878],
#   [0.49861944, 0.49947158, 0.50059104, 0.50023127]], dtype = tf.float32)

vec1 = tf.constant([[0.4978641 , 0.4956584 , 0.4949304 , 0.49439728, 0.49452817,
        0.4958884 , 0.4963342 , 0.49593773, 0.49512833, 0.49550587,
        0.49697465, 0.4975773 , 0.49848527, 0.4995232 , 0.49985358,
        0.49962163],
       [0.49549603, 0.48905388, 0.48708674, 0.4877914 , 0.48884934,
        0.48947975, 0.49131534, 0.49057013, 0.4885408 , 0.48942864,
        0.49258766, 0.49458462, 0.4973427 , 0.49927768, 0.5003911 ,
        0.49981767],
       [0.49481988, 0.4877937 , 0.48733968, 0.4927347 , 0.49941167,
        0.50185966, 0.50076365, 0.49616677, 0.49190122, 0.4903961 ,
        0.4916535 , 0.4947803 , 0.4989497 , 0.5014477 , 0.50265616,
        0.50091815],
       [0.49613094, 0.49168405, 0.49528345, 0.5066927 , 0.51926625,
        0.5290109 , 0.5255986 , 0.5159217 , 0.50637305, 0.49645182,
        0.49435067, 0.49748772, 0.5015879 , 0.50414276, 0.50530475,
        0.50302917],
       [0.49826103, 0.49734956, 0.5063491 , 0.5230674 , 0.5447407 ,
        0.5585421 , 0.5575325 , 0.5429373 , 0.52183753, 0.5054217 ,
        0.49832663, 0.49789354, 0.5008529 , 0.5051209 , 0.50628245,
        0.50336254],
       [0.49848902, 0.49875587, 0.5104595 , 0.5306728 , 0.5607434 ,
        0.57694983, 0.5740651 , 0.55711186, 0.5304589 , 0.5115793 ,
        0.49730125, 0.4946889 , 0.49860254, 0.50537866, 0.50765926,
        0.5034961 ],
       [0.49692908, 0.49479628, 0.5038091 , 0.52817476, 0.5567202 ,
        0.5766375 , 0.5714736 , 0.552144  , 0.5294191 , 0.5081695 ,
        0.49444747, 0.49119878, 0.49804845, 0.5044677 , 0.50759953,
        0.5038293 ],
       [0.49604037, 0.4896978 , 0.49505466, 0.51802427, 0.5386894 ,
        0.5555474 , 0.5553916 , 0.53705835, 0.5181776 , 0.4994972 ,
        0.48908502, 0.48629636, 0.49497905, 0.50344086, 0.5064416 ,
        0.50285274],
       [0.4959775 , 0.4886112 , 0.49028432, 0.50256294, 0.51662964,
        0.528004  , 0.5277405 , 0.5141205 , 0.5007169 , 0.4924641 ,
        0.4862668 , 0.4877291 , 0.49525216, 0.50327724, 0.5075504 ,
        0.50384295],
       [0.49701646, 0.49050507, 0.488937  , 0.49323085, 0.49995005,
        0.5067111 , 0.5054026 , 0.49548566, 0.48946118, 0.4891706 ,
        0.4925281 , 0.49593738, 0.50055325, 0.5059984 , 0.508257  ,
        0.50412536],
       [0.49876624, 0.4948571 , 0.49127162, 0.49093327, 0.4913554 ,
        0.49337223, 0.49405855, 0.4897231 , 0.48721927, 0.4937782 ,
        0.5014173 , 0.50512713, 0.5050719 , 0.5046365 , 0.50445545,
        0.5015145 ],
       [0.5013082 , 0.5007559 , 0.4990454 , 0.49523154, 0.4920312 ,
        0.49025568, 0.49094442, 0.4903319 , 0.4915165 , 0.5000091 ,
        0.5073812 , 0.50864124, 0.5036821 , 0.4995103 , 0.49828368,
        0.49889606],
       [0.5034798 , 0.50726914, 0.50701916, 0.50465554, 0.49810913,
        0.49325868, 0.49389797, 0.4949675 , 0.49890733, 0.50624317,
        0.5095126 , 0.5065519 , 0.4998414 , 0.49380058, 0.49281672,
        0.49675387],
       [0.5039327 , 0.5092165 , 0.51090914, 0.51043284, 0.50547206,
        0.4994874 , 0.4976094 , 0.5004631 , 0.50473547, 0.5099943 ,
        0.5088441 , 0.50153214, 0.49306548, 0.48713982, 0.48760238,
        0.49469545],
       [0.50362045, 0.5088228 , 0.51180565, 0.5127194 , 0.5098044 ,
        0.5039876 , 0.5009916 , 0.50176984, 0.50630695, 0.5080712 ,
        0.5039945 , 0.49686325, 0.49104849, 0.4872204 , 0.48728675,
        0.49425945],
       [0.5022053 , 0.50436574, 0.5055876 , 0.5068979 , 0.5054464 ,
        0.5024895 , 0.5010205 , 0.5010002 , 0.50180304, 0.50210136,
        0.50021565, 0.49727255, 0.4957635 , 0.49458537, 0.4945194 ,
        0.49758774]], dtype = tf.float32)

# vec1 = tf.constant([[1,1,1,0,0],
#                     [1,1,1,0,0],
#                     [1,1,1,0,0],
#                     [0,0,0,0,0], 
#                     [0,0,0,0,0]], dtype=tf.float32)
# # vec1 = tf.constant(np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]), dtype=tf.float32)

# vec2 = tf.constant([[0,0,0,0,0],
#                     [0,0,0,0,0], 
#                     [1,1,1,0,0],
#                     [1,1,1,0,0],
#                     [1,1,1,0,0]], dtype=tf.float32)

# vec3 = tf.constant([[0,0,0,0,0],
#                     [0,0,0,0,0], 
#                     [0,0,0,0,0],
#                     [0,0,0,0,0],
#                     [0,0,0,0,0]], dtype=tf.float32)

# vec1 = tf.constant([[0, 0, 0, 0],
#   [0, 0, 0, 0],
#   [0, 0, 0, 0],
#   [0, 0, 0, 1]])

# vec2 = tf.constant([[0, 0, 0, 0],
#   [0, 0, 0, 0],
#   [0, 0, 0, 0],
#   [0, 0, 0, 0]])

# orig_data = tf.convert_to_tensor(np.reshape(train_data, [2, 4,4,3]))
# print(mask_labels)
vec = tf.reshape(vec1, [1, 16,16])

batch_len = tf.shape(vec)[0]
input_shape = (img_shape[0], img_shape[1])
rows, cols = input_shape[0], input_shape[1]
item_shape = tf.Variable([item_size[0], item_size[1]])
d_rows, d_cols = item_shape[0], item_shape[1]
subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1

ii, jj = tf.meshgrid(tf.range(d_rows), tf.range(d_cols), indexing='ij')
d_ii, d_jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')

subm_ii = ii[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_ii
subm_jj = jj[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_jj

subm_st = tf.stack([subm_ii, subm_jj], axis=-1)

gather_exp = tf.map_fn(lambda mask: tf.gather_nd(mask, subm_st), vec)

subm_dims = tf.shape(gather_exp)
gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[1] * subm_dims[2], subm_dims[3], subm_dims[4]])
reduced_mat = tf.map_fn(lambda mask: tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(mask))[-1], gather_exp)
#     pred_crds = argmax_2d(reduced_mat, batch_len)

inp_tensor_shape = tf.shape(reduced_mat)
nimgs = inp_tensor_shape[0]
img_dims = inp_tensor_shape[1:]
img_len = inp_tensor_shape[-1]
flat_img = tf.reshape(reduced_mat, [-1, tf.reduce_prod(img_dims)])

# # argmax of the flat tensor
argmax_x = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) // tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))
argmax_y = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) % tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))

res = tf.stack([argmax_x, argmax_y, tf.zeros([batch_len], dtype=tf.float64)], axis = 1, name= "pred_crds")    

pred_crds = tf.cast(res, dtype = tf.int64, name= "predicted_crds")


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(vec))
#     print(sess.run(vec[0:1, :, :]))
#     print(sess.run(pred_crds[0:, :]))
    print(sess.run(pred_crds))
#     print(sess.run(res))


In [ ]:
"""Working Complete Network"""
# # Placeholders
# x = tf.placeholder(tf.float32, shape=[None, img_shape[0]*img_shape[1]*img_shape[2]], name='x')
# x_image = tf.reshape(x, [-1, img_shape[0], img_shape[1], img_shape[2]])

# mask_y_true = tf.placeholder(tf.float32, shape=[None, img_shape[0] * img_shape[1]], name='mask_y_true')
# mask_y_true_cls = tf.argmax(mask_y_true, axis=1) 

# sd_y_true = tf.placeholder(tf.float32, shape=[None, 2], name='sd_y_true')
# sd_y_true_cls = tf.argmax(sd_y_true, axis=1)        

# initializer = tf.contrib.layers.xavier_initializer()

# name_scope = 'train_mask'

# # mask_graph layer configurations
# m_filter_size0 = 16          # Convolution filters(kernel) are 4 x 4 pixels.
# m_num_filters0 = 16         # There are 16 of these filters.

# m_filter_size1 = 8          # Convolution filters are 4 x 4 pixels.
# m_num_filters1 = 16         # There are 16 of these filters.

# # Convolutional Layer 2.
# m_filter_size2 = 8          # Convolution filters are 2 x 2 pixels.
# m_num_filters2 = 16         # There are 16 of these filters.

# m_filter_size3 = 8          # Convolution filters are 2 x 2 pixels.
# m_num_filters3 = 4         # There are 4 of these filters.

# # Convolutional Layer 3.
# m_filter_size4 = 4          # Convolution filters are 2 x 2 pixels.
# m_num_filters4 = 32         # There are 32 of these filters.

# m_filter_size5 = 2          # Convolution filters are 2 x 2 pixels.
# m_num_filters5 = 16         # There are 16 of these filters.


# # Fully-connected layer.
# m_fc_size = 2000             # Number of neurons in fully-connected layer.


# """SD Network Layer Architecture"""
# filter_size1 = 4          # Convolution filters are 4 x 4 pixels.
# num_filters1 = 16         # There are 16 of these filters.

# # Convolutional Layer 2.
# filter_size2 = 2          # Convolution filters are 2 x 2 pixels.
# num_filters2 = 32         # There are 32 of these filters.

# # Convolutional Layer 3.
# filter_size3 = 2          # Convolution filters are 2 x 2 pixels.
# num_filters3 = 64         # There are 64 of these filters.

# # Convolutional Layer 4.
# filter_size4 = 2          # Convolution filters are 2 x 2 pixels.
# num_filters4 = 128         # There are 128 of these filters.

# # Fully-connected layer.
# fc_size = 256             # Number of neurons in fully-connected layer.

# with tf.name_scope(name_scope):
# # First Convolution Layer
#     layer_name = 'mask_conv_layer'
#     shape = [m_filter_size0, m_filter_size0, img_shape[2], m_num_filters0]
#     mask_weights = tf.Variable(initializer(shape), name=layer_name+'_W')  
#     mask_biases = tf.Variable(tf.constant(0.05, shape=[m_num_filters0]), name=layer_name+'_b')

#     layer0_conv0, weights_conv0 = new_conv_layer(input=x_image,
#                                                 num_input_channels=img_shape[2],
#                                                 filter_size=m_filter_size0,
#                                                 num_filters=m_num_filters0,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv1',
#                                                 use_pooling=True)

#     layer1_conv1, weights_conv1 = new_conv_layer(input=layer0_conv0,
#                                                 num_input_channels=m_num_filters0,
#                                                 filter_size=m_filter_size1,
#                                                 num_filters=m_num_filters1,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv2',
#                                                 use_pooling=True)

#     layer2_conv2, weights_conv2 =  new_conv_layer(input=layer1_conv1,
#                                                num_input_channels=m_num_filters1,
#                                                filter_size=m_filter_size2,
#                                                num_filters=m_num_filters2,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv3',
#                                                use_pooling=True)

#     layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
#                                                num_input_channels=m_num_filters2,
#                                                filter_size=m_filter_size3,
#                                                num_filters=m_num_filters3,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv4',
#                                                use_pooling=True)

#     layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
#                                                num_input_channels=m_num_filters3,
#                                                filter_size=m_filter_size4,
#                                                num_filters=m_num_filters4,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv5',
#                                                use_pooling=True)


#     layer5_conv5, weights_conv5 =  new_conv_layer(input=layer4_conv4,
#                                                num_input_channels=m_num_filters4,
#                                                filter_size=m_filter_size5,
#                                                num_filters=m_num_filters5,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv6',
#                                                use_pooling=True)

#     layer_flat, num_features = flatten_layer(layer5_conv5)
# #     print(layer_flat)
#     layer_fc1 = new_fc_layer(input=layer_flat,
#                              num_inputs=num_features,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc1',
#                              use_relu=True)
# #     print('layer_fc1:', layer_fc1)
#     layer_fc2 = new_fc_layer(input=layer_fc1,
#                              num_inputs=m_fc_size,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc2',
#                              use_relu=False)

#     layer_fc3 = new_fc_layer(input=layer_fc2,
#                              num_inputs=m_fc_size,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc3',
#                              use_relu=False)

#     mask_layer_fc4 = new_fc_layer(input=layer_fc3,
#                              num_inputs=m_fc_size,
#                              num_outputs=img_shape[0] * img_shape[1],
#                              name_scope = 'mask',
#                              layer_name = 'fc4',
#                              use_relu=False)

# #     mask_drop_out = tf.nn.dropout(mask_layer_fc4, 0.5, name="drop_out")
# #     y_pred = tf.nn.softmax(mask_drop_out, name="softmax_output")
#     y_pred = mask_layer_fc4
    
#     y_pred_norm = tf.to_int32(normalise(y_pred), name= layer_name + "/y_pred_norm")
#     y_pred_norm = tf.reshape(y_pred_norm, [-1, img_shape[0],img_shape[1]])

# #     orig_vec = tf.convert_to_tensor(np.reshape(train_data, [train_batch_size, img_shape[0],img_shape[1],img_shape[2]]))
# #     mask_vec = tf.convert_to_tensor(np.reshape(mask_labels, [train_batch_size, img_shape[0],img_shape[1]]))    
    
# #     Item Extraction
#     items = extract_patch(x_image, y_pred_norm)
    
#     mask_loss = tf.square(mask_y_true - y_pred)
#     mask_cost = tf.reduce_mean(mask_loss)

#     ### some more performance measures
#     mask_correct_prediction = tf.equal(y_pred, mask_y_true)
#     mask_accuracy = tf.reduce_mean(tf.cast(mask_correct_prediction, tf.float32))        

# name_scope = 'mask_sd'
# with tf.name_scope(name_scope):

#     input_sd = tf.cast(items, dtype = tf.float32, name=layer_name + "/input_sd")
    
#     layer1_conv1, weights_conv1 = new_conv_layer(input=input_sd,
#                                                 num_input_channels=img_shape[2],
#                                                 filter_size=filter_size1,
#                                                 num_filters=num_filters1,
#                                                  name_scope = 'mask_sd_graph',
#                                                  layer_name = 'conv1',
#                                                 use_pooling=True)

#     layer2_conv2, weights_conv2 = new_conv_layer(input=layer1_conv1,
#                                                 num_input_channels=num_filters1,
#                                                 filter_size=filter_size2,
#                                                 num_filters=num_filters2,
#                                                  name_scope = 'mask_sd_graph',
#                                                  layer_name = 'conv2',
#                                                 use_pooling=True)

#     layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
#                                                num_input_channels=num_filters2,
#                                                filter_size=filter_size3,
#                                                num_filters=num_filters3,
#                                                  name_scope = 'mask_sd_graph',
#                                                  layer_name = 'conv3',
#                                                use_pooling=True)

#     layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
#                                                num_input_channels=num_filters3,
#                                                filter_size=filter_size4,
#                                                num_filters=num_filters4,
#                                                  name_scope = 'mask_sd_graph',
#                                                  layer_name = 'conv4',
#                                                use_pooling=True)

#     layer_flat, num_features = flatten_layer(layer4_conv4)       

#     layer_fc1 = new_fc_layer(input=layer_flat,
#                      num_inputs=num_features,
#                      num_outputs=fc_size,
#                      name_scope = 'mask_sd_graph',
#                      layer_name = 'fc1',
#                      use_relu=True)

#     layer_fc2 = new_fc_layer(input=layer_fc1,
#                              num_inputs=fc_size,
#                              num_outputs=fc_size,
#                              name_scope = 'mask_sd_graph',
#                              layer_name = 'fc2',
#                              use_relu=False)

#     layer_fc3 = new_fc_layer(input=layer_fc2,
#                              num_inputs=fc_size,
#                              num_outputs=fc_size,
#                              name_scope = 'mask_sd_graph',
#                              layer_name = 'fc3',
#                              use_relu=False)

#     layer_fc4 = new_fc_layer(input=layer_fc3,
#                              num_inputs=fc_size,
#                              num_outputs=num_classes,
#                              name_scope = 'mask_sd_graph',
#                              layer_name = 'fc4',
#                              use_relu=False)


    
    
#     sd_y_pred = tf.nn.softmax(layer_fc4, name=layer_name + "/sd_y_pred")
#     sd_y_pred_cls = tf.argmax(sd_y_pred, axis=1)
    
#     sd_loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=sd_y_pred, labels=sd_y_true)
    
#     train_op1 = tf.train.AdamOptimizer().minimize(mask_loss)
#     train_op2 = tf.train.AdamOptimizer().minimize(sd_loss)
#     final_train_op = tf.group(train_op1, train_op2)    

#     sd_cost = tf.reduce_mean(sd_loss)
    
#     sd_correct_prediction = tf.equal(sd_y_pred_cls, sd_y_true_cls)
#     sd_accuracy = tf.reduce_mean(tf.cast(sd_correct_prediction, tf.float32))        
    

In [ ]:
# # Placeholders
# x = tf.placeholder(tf.float32, shape=[None, img_shape[0]*img_shape[1]*img_shape[2]], name='x')
# x_image = tf.reshape(x, [-1, img_shape[0], img_shape[1], img_shape[2]])

# mask_y_true = tf.placeholder(tf.float32, shape=[None, img_shape[0] * img_shape[1]], name='mask_y_true')
# mask_y_true_cls = tf.argmax(mask_y_true, axis=1) 

# sd_y_true = tf.placeholder(tf.float32, shape=[None, 2], name='sd_y_true')
# sd_y_true_cls = tf.argmax(sd_y_true, axis=1)        

# initializer = tf.contrib.layers.xavier_initializer()

# name_scope = 'train_mask'

# # mask_graph layer configurations
# # mask_graph layer configurations
# m_filter_size0 = 16          # Convolution filters(kernel) are 4 x 4 pixels.
# m_num_filters0 = 16         # There are 16 of these filters.

# m_filter_size1 = 8          # Convolution filters are 4 x 4 pixels.
# m_num_filters1 = 16         # There are 16 of these filters.

# # Convolutional Layer 2.
# m_filter_size2 = 8          # Convolution filters are 2 x 2 pixels.
# m_num_filters2 = 16         # There are 16 of these filters.

# m_filter_size3 = 8          # Convolution filters are 2 x 2 pixels.
# m_num_filters3 = 4         # There are 4 of these filters.

# # Convolutional Layer 3.
# m_filter_size4 = 4          # Convolution filters are 2 x 2 pixels.
# m_num_filters4 = 32         # There are 32 of these filters.

# m_filter_size5 = 2          # Convolution filters are 2 x 2 pixels.
# m_num_filters5 = 16         # There are 16 of these filters.


# # Fully-connected layer.
# m_fc_size = 2000             # Number of neurons in fully-connected layer.


# """SD Network Layer Architecture"""
# filter_size1 = 4          # Convolution filters are 4 x 4 pixels.
# num_filters1 = 16         # There are 16 of these filters.

# # Convolutional Layer 2.
# filter_size2 = 2          # Convolution filters are 2 x 2 pixels.
# num_filters2 = 32         # There are 32 of these filters.

# # Convolutional Layer 3.
# filter_size3 = 2          # Convolution filters are 2 x 2 pixels.
# num_filters3 = 64         # There are 64 of these filters.

# # Convolutional Layer 4.
# filter_size4 = 2          # Convolution filters are 2 x 2 pixels.
# num_filters4 = 128         # There are 128 of these filters.

# # Fully-connected layer.
# fc_size = 256             # Number of neurons in fully-connected layer.

# with tf.name_scope(name_scope):
# # First Convolution Layer
#     layer_name = 'mask_conv_layer'
#     shape = [m_filter_size0, m_filter_size0, img_shape[2], m_num_filters0]
#     mask_weights = tf.Variable(initializer(shape), name=layer_name+'_W')  
#     mask_biases = tf.Variable(tf.constant(0.05, shape=[m_num_filters0]), name=layer_name+'_b')

#     layer0_conv0, weights_conv0 = new_conv_layer(input=x_image,
#                                                 num_input_channels=img_shape[2],
#                                                 filter_size=m_filter_size0,
#                                                 num_filters=m_num_filters0,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv1',
#                                                 use_pooling=True)
# #     layer0_conv0_drpout = tf.nn.dropout(layer0_conv0, 0.3, name="drop_out")

#     layer1_conv1, weights_conv1 = new_conv_layer(input=layer0_conv0,
#                                                 num_input_channels=m_num_filters0,
#                                                 filter_size=m_filter_size1,
#                                                 num_filters=m_num_filters1,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv2',
#                                                 use_pooling=True)

# #     layer1_conv1_drpout = tf.nn.dropout(layer1_conv1, 0.3, name="drop_out")

#     layer2_conv2, weights_conv2 =  new_conv_layer(input=layer1_conv1,
#                                                num_input_channels=m_num_filters1,
#                                                filter_size=m_filter_size2,
#                                                num_filters=m_num_filters2,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv3',
#                                                use_pooling=True)

#     layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
#                                                num_input_channels=m_num_filters2,
#                                                filter_size=m_filter_size3,
#                                                num_filters=m_num_filters3,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv4',
#                                                use_pooling=True)

#     layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
#                                                num_input_channels=m_num_filters3,
#                                                filter_size=m_filter_size4,
#                                                num_filters=m_num_filters4,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv5',
#                                                use_pooling=True)


#     layer5_conv5, weights_conv5 =  new_conv_layer(input=layer4_conv4,
#                                                num_input_channels=m_num_filters4,
#                                                filter_size=m_filter_size5,
#                                                num_filters=m_num_filters5,
#                                                  name_scope = 'mask',
#                                                  layer_name = 'conv6',
#                                                use_pooling=True)
    
# #     layer6_conv6, weights_conv6 =  new_conv_layer(input=layer5_conv5,
# #                                            num_input_channels=m_num_filters5,
# #                                            filter_size=m_filter_size6,
# #                                            num_filters=m_num_filters6,
# #                                              name_scope = 'mask',
# #                                              layer_name = 'conv7',
# #                                            use_pooling=True)

#     layer_flat, num_features = flatten_layer(layer5_conv5)
# #     print(layer_flat)
#     layer_fc1 = new_fc_layer(input=layer_flat,
#                              num_inputs=num_features,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc1',
#                              use_relu=False)
# #     print('layer_fc1:', layer_fc1)
#     layer_fc2 = new_fc_layer(input=layer_fc1,
#                              num_inputs=m_fc_size,
#                              num_outputs=m_fc_size,
#                              name_scope = 'mask',
#                              layer_name = 'fc2',
#                              use_relu=False)

# #     layer_fc3 = new_fc_layer(input=layer_fc2,
# #                              num_inputs=m_fc_size,
# #                              num_outputs=m_fc_size,
# #                              name_scope = 'mask',
# #                              layer_name = 'fc3',
# #                              use_relu=False)
# #     mask_drop_out = tf.nn.dropout(layer_fc3, 0.5, name="drop_out")

#     mask_layer_fc3 = new_fc_layer(input=layer_fc2,
#                              num_inputs=m_fc_size,
#                              num_outputs=img_shape[0] * img_shape[1],
#                              name_scope = 'mask',
#                              layer_name = 'fc3',
#                              use_relu=False)

# #     mask_drop_out = tf.nn.dropout(mask_layer_fc4, 0.5, name="drop_out")
# #     y_pred = tf.nn.softmax(mask_drop_out, name="softmax_output")
#     y_pred = mask_layer_fc3
    
#     # round numbers less than 0.5 to zero;
#     # by making them negative and taking the maximum with 0
# #     differentiable_round = tf.maximum(y_pred-0.499,0)
#     # scale the remaining numbers (0 to 0.5) to greater than 1
#     # the other half (zeros) is not affected by multiplication
# #     differentiable_round = differentiable_round * 10000
#     # take the minimum with 1
# #     y_pred_round = tf.minimum(differentiable_round, 1)
# #     print(differentiable_round)

#     y_pred_sigmoid = tf.nn.sigmoid(y_pred, name="sigmoid_output")
#     y_pred_sigmoid = tf.reshape(y_pred_sigmoid, [-1, img_shape[0], img_shape[1]])
    
#     batch_len = tf.shape(y_pred_sigmoid)[0]
#     input_shape = (img_shape[0], img_shape[1])
#     rows, cols = input_shape[0], input_shape[1]
#     item_shape = tf.Variable([item_size[0], item_size[1]])
#     d_rows, d_cols = item_shape[0], item_shape[1]
#     subm_rows, subm_cols = rows - d_rows + 1, cols - d_cols + 1

#     ii, jj = tf.meshgrid(tf.range(d_rows), tf.range(d_cols), indexing='ij')
#     d_ii, d_jj = tf.meshgrid(tf.range(subm_rows), tf.range(subm_cols), indexing='ij')

#     subm_ii = ii[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_ii
#     subm_jj = jj[:subm_rows, :subm_cols, tf.newaxis, tf.newaxis] + d_jj

#     subm_st = tf.stack([subm_ii, subm_jj], axis=-1)
    
#     gather_exp = tf.map_fn(lambda mask: tf.gather_nd(mask, subm_st), y_pred_sigmoid)

#     subm_dims = tf.shape(gather_exp)
#     gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[1] * subm_dims[2], subm_dims[3], subm_dims[4]])
#     reduced_mat = tf.map_fn(lambda mask: tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(mask))[-1], gather_exp)
    
#     inp_tensor_shape = tf.shape(reduced_mat)
#     nimgs = inp_tensor_shape[0]
#     img_dims = inp_tensor_shape[1:]
#     img_len = inp_tensor_shape[-1]
#     flat_img = tf.reshape(reduced_mat, [-1, tf.reduce_prod(img_dims)])

#     # # argmax of the flat tensor
#     argmax_x = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) // tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))
#     argmax_y = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) % tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))

#     res = tf.stack([argmax_x, argmax_y, tf.zeros([batch_len], dtype=tf.float64)], axis = 1, name= "pred_crds1")    
    
#     pred_crds = tf.cast(res, dtype = tf.int64, name= "predicted_crds")    
    
#     itms = tf.map_fn(lambda idx: tf.cast(tf.slice(x_image[tf.cast(idx, tf.int64), :, :, :],pred_crds[tf.cast(idx, tf.int64), :], [item_size[0],item_size[1], 3]), dtype = tf.float64), tf.cast(tf.range(batch_len), dtype = tf.float64), name= "original_items")


#     mask_loss = tf.square(mask_y_true - y_pred)
#     mask_cost = tf.reduce_mean(mask_loss)
    
#     train_op1 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(mask_loss)

# #     y_pred = tf.reshape(y_pred, [-1, img_shape[0] * img_shape[1]])
# #     y_pred_round = tf.reshape(y_pred_round, [-1, img_shape[0], img_shape[1]])
    
# #     items = extract_patch(x_image, y_pred_sigmoid)


# #     mask_loss = tf.reduce_sum(tf.map_fn(lambda idx: tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred_ind[:,tf.cast(idx, dtype = tf.int32),:], labels=mask_y_true[:, tf.cast(idx, dtype = tf.int32), :]), tf.cast(tf.range(img_shape[0] * img_shape[1]), dtype = tf.float32)))
# #     print(mask_loss)
# #     mask_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred_round, labels=mask_y_true)


# #     gather_exp = tf.map_fn(lambda mask: tf.gather_nd(mask, subm_st), y_pred_sigmoid)

# #     subm_dims = tf.shape(gather_exp)
# #     gather_exp = tf.reshape(gather_exp, [-1,  subm_dims[1] * subm_dims[2], subm_dims[3], subm_dims[4]])
# #     reduced_mat = tf.map_fn(lambda mask: tf.scan(lambda a, b: tf.multiply(a, b), tf.squeeze(mask))[-1], gather_exp)
# # #     pred_crds = argmax_2d(reduced_mat, batch_len)
    
# #     inp_tensor_shape = tf.shape(reduced_mat)
# #     nimgs = inp_tensor_shape[0]
# #     img_dims = inp_tensor_shape[1:]
# #     img_len = inp_tensor_shape[-1]
# #     flat_img = tf.reshape(reduced_mat, [-1, tf.reduce_prod(img_dims)])

# #     # # argmax of the flat tensor
# #     argmax_x = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) // tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))
# #     argmax_y = tf.map_fn(lambda img: tf.cast(tf.argmax(img), tf.float64) % tf.cast(img_len, dtype=tf.float64), tf.cast(flat_img, dtype=tf.float64))

# #     res = tf.stack([argmax_x, argmax_y, tf.zeros([batch_len], dtype=tf.float64)], axis = 1, name= "pred_crds")    
    
# #     pred_crds = tf.cast(res, dtype = tf.int64, name= "predicted_crds")    
    
    
#     ### some more performance measures
#     mask_correct_prediction = tf.equal(y_pred, mask_y_true)
#     mask_accuracy = tf.reduce_mean(tf.cast(mask_correct_prediction, tf.float32))        

# # name_scope = 'mask_sd'
# # with tf.name_scope(name_scope):

# #     input_sd = tf.cast(items, dtype = tf.float32, name=layer_name + "/input_sd")
    
# #     layer1_conv1, weights_conv1 = new_conv_layer(input=input_sd,
# #                                                 num_input_channels=img_shape[2],
# #                                                 filter_size=filter_size1,
# #                                                 num_filters=num_filters1,
# #                                                  name_scope = 'mask_sd_graph',
# #                                                  layer_name = 'conv1',
# #                                                 use_pooling=True)

# #     layer2_conv2, weights_conv2 = new_conv_layer(input=layer1_conv1,
# #                                                 num_input_channels=num_filters1,
# #                                                 filter_size=filter_size2,
# #                                                 num_filters=num_filters2,
# #                                                  name_scope = 'mask_sd_graph',
# #                                                  layer_name = 'conv2',
# #                                                 use_pooling=True)

# #     layer3_conv3, weights_conv3 =  new_conv_layer(input=layer2_conv2,
# #                                                num_input_channels=num_filters2,
# #                                                filter_size=filter_size3,
# #                                                num_filters=num_filters3,
# #                                                  name_scope = 'mask_sd_graph',
# #                                                  layer_name = 'conv3',
# #                                                use_pooling=True)

# #     layer4_conv4, weights_conv4 =  new_conv_layer(input=layer3_conv3,
# #                                                num_input_channels=num_filters3,
# #                                                filter_size=filter_size4,
# #                                                num_filters=num_filters4,
# #                                                  name_scope = 'mask_sd_graph',
# #                                                  layer_name = 'conv4',
# #                                                use_pooling=True)

# #     layer_flat, num_features = flatten_layer(layer4_conv4)       

# #     layer_fc1 = new_fc_layer(input=layer_flat,
# #                      num_inputs=num_features,
# #                      num_outputs=fc_size,
# #                      name_scope = 'mask_sd_graph',
# #                      layer_name = 'fc1',
# #                      use_relu=True)

# #     layer_fc2 = new_fc_layer(input=layer_fc1,
# #                              num_inputs=fc_size,
# #                              num_outputs=fc_size,
# #                              name_scope = 'mask_sd_graph',
# #                              layer_name = 'fc2',
# #                              use_relu=False)

# #     layer_fc3 = new_fc_layer(input=layer_fc2,
# #                              num_inputs=fc_size,
# #                              num_outputs=fc_size,
# #                              name_scope = 'mask_sd_graph',
# #                              layer_name = 'fc3',
# #                              use_relu=False)

# #     layer_fc4 = new_fc_layer(input=layer_fc3,
# #                              num_inputs=fc_size,
# #                              num_outputs=num_classes,
# #                              name_scope = 'mask_sd_graph',
# #                              layer_name = 'fc4',
# #                              use_relu=False)


    
    
# #     sd_y_pred = tf.nn.softmax(layer_fc4, name=layer_name + "/sd_y_pred")
# #     sd_y_pred_cls = tf.argmax(sd_y_pred, axis=1)
    
# #     sd_loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=sd_y_pred, labels=sd_y_true)
    
# #     train_op1 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(mask_loss)
# #     train_op2 = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(sd_loss)
# #     final_train_op = tf.group(train_op1, train_op2)    

# #     sd_cost = tf.reduce_mean(sd_loss)
    
# #     sd_correct_prediction = tf.equal(sd_y_pred_cls, sd_y_true_cls)
# #     sd_accuracy = tf.reduce_mean(tf.cast(sd_correct_prediction, tf.float32))        
    